# Wikipics demo

In [ ]:
from pathlib import Path
from typing import NamedTuple

from IPython.display import display
from ipywidgets.widgets import Image, Layout
import httpx

from wikipics import get_sample_url, get_sample_urls

class ImageRecord(NamedTuple):
    pixels: bytes
    name: str
    size: int

In [ ]:
def fetch(url) -> ImageRecord:
    resp = httpx.get(url)
    resp.raise_for_status()
    name = Path(url).name
    return ImageRecord(resp.content, name, len(resp.content))

In [ ]:
url = get_sample_url(1_000_000)
img_rec = fetch(url)
print(f'{img_rec.size:12_} bytes | {img_rec.name}')
Image(value=img_rec.pixels)

In [ ]:
%%time
img_widgets = []
qty = 10
urls = get_sample_urls(2_000_000, qty)
total_bytes = 0

for url in urls:
    img_rec = fetch(url)
    total_bytes += img_rec.size
    display(Image(value=img_rec.pixels, layout=Layout(width='20%')))
    print(f'{img_rec.size:12_} bytes | {img_rec.name}')

print(f'TOTAL BYTES: {total_bytes:_}')

In [ ]:
from gallery import Gallery

In [ ]:
%%timed
num_images = 10
gallery = Gallery(num_images)
gallery.display()

urls = get_sample_urls(2_000_000, num_images)

for i, url in enumerate(urls):
    img_rec = fetch(url)
    total_bytes += img_rec.size
    gallery.update(i, img_rec.pixels, url)

print(f'TOTAL BYTES: {gallery.size:_}')

In [5]:
l = 'banana maçã pêra uva'.split()
list(map(str.upper, l))

['BANANA', 'MAÇÃ', 'PÊRA', 'UVA']

In [ ]:
%%time
from concurrent import futures

num_images = 10
gallery = Gallery(num_images)
gallery.display()

urls = get_sample_urls(2_000_000, num_images)
with futures.ThreadPoolExecutor() as pool:
    img_records = pool.map(fetch, urls)
    for i, img_rec in enumerate(img_records):
        gallery.update(i, img_rec.pixels, img_rec.name)

print(f'TOTAL BYTES: {gallery.size:_}')

In [ ]:
%%time
from concurrent import futures

num_images = 10
gallery = Gallery(num_images)
gallery.display()
urls = get_sample_urls(2_000_000, num_images)

with futures.ThreadPoolExecutor() as pool:
    tasks = [pool.submit(fetch, url) for url in urls]
    for i, future in enumerate(futures.as_completed(tasks)):
        img_rec = future.result()
        gallery.update(i, img_rec.pixels, img_rec.name)
        
print(f'TOTAL BYTES: {gallery.size:_}')